# Webscraper

Import of the usual libraries in addition the  HTTP library  ([request](https://docs.python-requests.org/en/latest/)) and the HTML/XML parser ([Beautifulsoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)). 

If you don't have a VPN, this API might be interesting ([OpenVPN](https://pypi.org/project/openvpn-api/))

The google.cloud import with bigquery and the key is just to get the article_id.

In [49]:
from google.cloud import bigquery

import pandas as pd
import numpy as np
from tqdm import tqdm
from bs4 import BeautifulSoup
import requests

import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="./key/key.json"

Connecting to bigquery

In [50]:
 %reload_ext google.cloud.bigquery

SQL-Query

In [51]:
   %%bigquery crawl

SELECT 
COUNT(*) article_count,
trans.article_id,
AVG(price) average_price
FROM`data-buckeye-347917.fashion_recommender.transaction` AS trans
GROUP BY trans.article_id
ORDER BY article_count DESC
LIMIT 1000;


Downloading: 100%|██████████| 1000/1000 [00:01<00:00, 710.41rows/s]


In [52]:
df = pd.DataFrame(crawl)

In [53]:
df.head()

,article_count,article_id,average_price
0,50287,706016001,0.032448
1,35043,706016002,0.032426
2,31718,372860001,0.012958
3,30199,610776002,0.008083
4,26329,759871002,0.005605


Set the articöe_id to a string and ad a 0 to it 

In [54]:
df["article_id"]=df["article_id"].astype(str)
df['article_id'] = df['article_id'].apply(lambda x: x.zfill(10))

Header is needed for sending the webside, "who" you are. 
And an url is given for test purposes.
They are set up in the response from requests.get(X)
the second is the parser to "read" the HTML stuff and translate it for humans.

In [55]:
headers = {'User-Agent':'Mozilla/15.0 (Macintosh;   Mac OS X 11_2_4) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/501.3.9'}
url = ''

response=requests.get(url,headers=headers)


soup=BeautifulSoup(response.text, 'html.parser')

Get an overview over the whole side and search for "price regular"

In [56]:
for item in soup.select('.hm-product-item'):
	try:
		print('----------------------------------------')
		print(item)



	except Exception as e:
		#raise e
		print('')

----------------------------------------
<article class="hm-product-item" data-articlecode="0706016001" data-brand="H&amp;M" data-category="ladies_jeans_skinny_skinnyhigh" data-energy-interval="" data-pre-access-end-date="" data-pre-access-groups="" data-pre-access-start-date="" data-style-with-articlecodes="" onclick="setVCParameter('SEARCH','0706016001'); setNotificationTicket('Oy9zZWFyY2gvc2VhcmNoLWhpdHMtd2l0aC1jb3VudC9zZWFyY2gtaGl0czsjO3Byb2R1Y3Rfa2V5OzA3MDYwMTZfZ3JvdXBfMDAxX2RlX2RlOzA3MDYwMTYwMDFfZGVfZGU7O05PTkU6Tk9ORTs0ODs','0706016001'); setOsaParameters(utag_data.category_id,'SMALL','0706016001');">
<div class="image-container">
<a class="item-link" href="/de_de/productpage.0706016001.html" title="Super Skinny High Jeans">
<img alt="Super Skinny High Jeans" class="item-image" data-altimage="//lp2.hm.com/hmgoepprod?set=source[/7c/6e/7c6ed6bf87f52c540d8d56b591a99f62557ad928.jpg],origin[dam],category[ladies_jeans_skinny_skinnyhigh],type[DESCRIPTIVESTILLLIFE],res[m],hmver[1]&amp;ca

In [57]:
for item in soup.select('.hm-product-item'):
	try:
		print('----------------------------------------')
		#print(item.select(".item-price")[0].get_text().strip())
		print(i, soup.find('span', {'class': 'price regular'}).text)


	except Exception as e:
		#raise e
		print('')

----------------------------------------
0372860001 19,99 €


We want this: ```<span class="price regular">19,99 €</span> ```


```{HTML}
</h3><strong class="item-price">
<span class="price regular">19,99 €</span> 
```

so we tell the parser... 

```{python}
case = {'article_id': i, 'price': soup.find('span', {'class': 'price regular'}).text[:-1]}
```
Searches for "price regular" in "span" and removes the euro sign. 

This is combined in a slow for-loop to have 2 requests a second. This is slowed on purpose, you can extract the request part and iterated ouside of the loop, which increases the speed dramatically but also your visuability. 
 Here .head(5) is added, to try the first 5 lines.

In [58]:
#response=requests.get(url,headers=headers)

sample = []

for i in tqdm(df["article_id"].head(5)):
	url = '{}'.format(i) 
	soup=BeautifulSoup(response.content,'lxml')
	response=requests.get(url,headers=headers)
	for item in soup.select('.hm-product-item'):
		try:
			print('----------------------------------------')
			case = {'article_id': i, 'price': soup.find('span', {'class': 'price regular'}).text[:-1]}
			sample.append(case)



		except Exception as e:
			#raise e
			print('')


 20%|██        | 1/5 [00:02<00:08,  2.25s/it]

----------------------------------------


 20%|██        | 1/5 [01:03<04:12, 63.23s/it]


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
sample

[{'article_id': '0706016001', 'price': '19,99 '},
 {'article_id': '0706016002', 'price': '19,99 '},
 {'article_id': '0372860001', 'price': '19,99 '},
 {'article_id': '0610776002', 'price': '7,99 '},
 {'article_id': '0759871002', 'price': '4,99 '}]

A new dataframe is formed from the sample list

In [ ]:
df_2 = pd.DataFrame(sample, columns=["article_id", "new_price"])


Merged

In [37]:
df=pd.merge(df,df_2, on=["article_id"], how="outer")

In [39]:
df_2["article_id"]=df_2["article_id"].astype(int)

In [40]:
#df_2.to_parquet("../parquet/articles_prices_right_type.parquet")

In [41]:
df_2["new_price"]=df_2['new_price'].apply(lambda x:  str(x).replace(',','.'))

In [42]:
df_2.replace(to_replace="None", value=np.nan, inplace=True)

In [43]:
#df_2["new_price"].fillna((df_2["average_price"]*10), inplace=True)

KeyError: 'average_price'

In [44]:
df_2["new_price"]=df_2["new_price"].astype(float)

In [45]:
df_2["total"]=(df_2["article_count"]*df_2["new_price"])

KeyError: 'article_count'

,article_id,new_price
0,706016001,NaN
1,706016002,NaN
2,372860001,NaN
3,610776002,NaN
4,759871002,NaN
